# Extracting insights from SQL queries using DuckDB
### *Author: Hank Hoang*
### *Date created: 03/07/2023*
### *Date finished: 04/07/2023*

## Import Libraries

In [2]:
import pandas as pd
import duckdb
connection = duckdb.connect()

In [3]:
#Reading the data
file_path = '/content/drive/MyDrive/IMDb_data.csv'
df = pd.read_csv(file_path)

<ipython-input-3-ced02e518387>:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [4]:
print(df.dtypes)

tconst              object
Type                object
Title               object
isAdult              int64
startYear           object
endYear             object
Runtime             object
Genres              object
Rating             float64
Number of Votes      int64
Director Name       object
dtype: object


### Rename Number of Votes column to call in SQL query

In [73]:
column_name_mapping = {'Number of Votes': 'NumberofVotes','Director Name':'DirectorName'}

# Use the rename() method on the DataFrame
df.rename(columns=column_name_mapping, inplace=True)

## Subset data without N/A Values in Runtime column

In [74]:
subset_df = df[~df['Runtime'].str.contains('N')]
# Convert the 'Runtime' column to integer
subset_df['Runtime'] = subset_df['Runtime'].astype(int)


<ipython-input-74-5e017314edfc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['Runtime'] = subset_df['Runtime'].astype(int)


## Implement SQL queries with DuckDB

In [11]:
# DuckDB connection
con = duckdb.connect(database=":memory:", read_only=False)

In [12]:
# Making SQL query for example
con.execute("SELECT Title FROM df ")
result = con.fetchdf()

In [13]:
# Select Title column
result

,Title
0,Carmencita
1,Le clown et ses chiens
2,Pauvre Pierrot
3,Un bon bock
4,Blacksmith Scene
...,...
944650,Pink Taxi
944651,Kuambil Lagi Hatiku
944652,My Sweet Prince
944653,The Trial of Joan Collins


## *Q1: How many movies have rating above than 7?*





In [17]:
con.execute("SELECT COUNT(*) AS RESULT FROM df WHERE Rating > 7;")
result = con.fetchdf()
result

,RESULT
0,482608


## *Q2: How many movies have rating above than 7 less than 30 minutes  ?*


In [16]:
con.execute("SELECT COUNT(*) AS RESULT FROM subset_df WHERE Rating > 7 and Runtime < 30 ;")
result = con.fetchdf()
result

,RESULT
0,128693


## *Q3: How many terrible movies( have rating less than 5)?*

In [18]:
con.execute("SELECT COUNT(*) AS RESULT FROM subset_df WHERE Rating < 5 ;")
result = con.fetchdf()
result

,RESULT
0,66579


## *Q4: Which Romance movie has the highest rating?*




In [35]:
con.execute("SELECT * FROM subset_df WHERE rating = (SELECT MAX(rating) FROM subset_df WHERE Genres LIKE '%Romance%');"  )
result = con.fetchdf()
result

,tconst,Type,Title,isAdult,startYear,endYear,Runtime,Genres,Rating,NumberofVotes,Director Name
0,tt0127236,video,Renegades 2,1,1995,\N,91,Adult,10.0,23,Buck Adams
1,tt0258117,short,Sally in Our Alley,0,1916,\N,40,"Romance,Short",10.0,12,Laurence Trimble
2,tt0270927,tvMovie,Das Ende vom Anfang,0,1963,\N,43,Comedy,10.0,11,Erik Ode
3,tt0281732,short,Closed for Business,0,1997,\N,4,Short,10.0,11,James H. Brown
4,tt0295190,tvMovie,Autobiografija utopljenice,0,1964,\N,90,Drama,10.0,12,Ivan Hetrich
...,...,...,...,...,...,...,...,...,...,...,...
774,tt9226836,short,Summer Rain,0,2017,\N,20,"Drama,Short",10.0,5,Miwako Van Weyenberg
775,tt9556070,short,Triple A,0,2022,\N,15,"Comedy,Drama,Romance",10.0,5,J.B. Rutagarama
776,tt9614362,tvSeries,Austintatious,0,2017,\N,22,"Comedy,Drama",10.0,5,Austin Greene
777,tt9646734,short,Dying Out Loud,0,2019,\N,20,"Documentary,Short",10.0,77,Jake J. Smith


## *Q5: 20 Romance movies has the highest rating and Number of Votes more than 50*


In [36]:
con.execute("SELECT * FROM subset_df WHERE Genres LIKE '%Romance%' AND NumberofVotes > 50  ORDER BY Rating DESC LIMIT 20;")
result = con.fetchdf()
result

,tconst,Type,Title,isAdult,startYear,endYear,Runtime,Genres,Rating,NumberofVotes,Director Name
0,tt4678496,short,Thanks for Accepting Me,0,2015,\N,25,"Romance,Short",10.0,162,Nikos Kourous
1,tt15513206,movie,Nee Jathaga,0,2021,\N,113,"Comedy,Drama,Romance",10.0,741,Bamidipati Veera
2,tt16282202,tvEpisode,Episode #1.1,0,2022,\N,44,"Comedy,Drama,Romance",10.0,55,Hadrah Daeng Ratu
3,tt16282212,tvEpisode,Episode #1.3,0,2022,\N,39,"Comedy,Drama,Romance",9.9,63,Hadrah Daeng Ratu
4,tt9044564,tvEpisode,Episode #1.2,0,2018,\N,24,"Drama,Romance",9.9,63,Muzammil Desai
5,tt14196560,tvEpisode,Episode #1.4,0,2021,\N,50,"Drama,Romance",9.8,55,Monty Tiwa
6,tt1968955,tvEpisode,I Can't Get Over My First Love,0,2011,\N,60,"Documentary,Romance",9.8,55,Jordan Ross
7,tt7717890,tvEpisode,Wrong Number,0,2017,\N,13,"Comedy,Romance",9.8,68,Aijaz Alladitta Shaikh
8,tt8422000,short,Half Girlfriend,0,2014,\N,10,"Romance,Short",9.8,91,V. Jayashankarr
9,tt0583452,tvEpisode,The One Where Everybody Finds Out,0,1999,\N,22,"Comedy,Romance",9.7,12282,Michael Lembeck


## *Q6: Top 10 movies has the highest votes and their respective ratings?*


In [38]:
con.execute("SELECT * FROM subset_df ORDER BY NumberofVotes DESC LIMIT 10;")
result = con.fetchdf()
result

,tconst,Type,Title,isAdult,startYear,endYear,Runtime,Genres,Rating,NumberofVotes,Director Name
0,tt0111161,movie,The Shawshank Redemption,0,1994,\N,142,Drama,9.3,2757004,Frank Darabont
1,tt0468569,movie,The Dark Knight,0,2008,\N,152,"Action,Crime,Drama",9.0,2729988,Christopher Nolan
2,tt1375666,movie,Inception,0,2010,\N,148,"Action,Adventure,Sci-Fi",8.8,2422670,Christopher Nolan
3,tt0137523,movie,Fight Club,0,1999,\N,139,Drama,8.8,2195170,David Fincher
4,tt0109830,movie,Forrest Gump,0,1994,\N,142,"Drama,Romance",8.8,2144463,Robert Zemeckis
5,tt0110912,movie,Pulp Fiction,0,1994,\N,154,"Crime,Drama",8.9,2117022,Quentin Tarantino
6,tt0816692,movie,Interstellar,0,2014,\N,169,"Adventure,Drama,Sci-Fi",8.7,1923978,Christopher Nolan
7,tt0120737,movie,The Lord of the Rings: The Fellowship of the Ring,0,2001,\N,178,"Action,Adventure,Drama",8.8,1921752,Peter Jackson
8,tt0068646,movie,The Godfather,0,1972,\N,175,"Crime,Drama",9.2,1918388,Francis Ford Coppola
9,tt0167260,movie,The Lord of the Rings: The Return of the King,0,2003,\N,201,"Action,Adventure,Drama",9.0,1893267,Peter Jackson


## *Q7: Duration for short Movies in average?*

In [50]:
con.execute("SELECT AVG(Runtime) AS average_value FROM subset_df WHERE Type = 'short';")
result = con.fetchdf()
result

,average_value
0,14.314315


**Short Movies often goes around 14.3 minutes**

## *Q8: Genres go with Adult movies?*



In [61]:
con.execute("SELECT Genres FROM df WHERE isAdult = 1 AND Genres NOT LIKE '%N%';")
result = con.fetchdf()
value_counts = result['Genres'].value_counts().head(10)
value_counts

Adult                   11368
Adult,Comedy              942
Adult,Romance             600
Adult,Drama               596
Adult,Drama,Romance       508
Adult,Crime,Drama         190
Adult,Comedy,Romance      161
Adult,Short               142
Adult,Comedy,Fantasy      126
Adult,Horror              122
Name: Genres, dtype: int64

## *Q9: Average Rating of Adult movies?*


In [66]:
con.execute("SELECT AVG(Rating) AS Average_value FROM subset_df WHERE isAdult = 1 ;")
result = con.fetchdf()
result

,Average_value
0,6.196387


## *Q10: Top 10 Directors has highest rating for their comedy movies?*

In [76]:
con.execute("SELECT DirectorName  FROM subset_df WHERE Genres LIKE '%Comedy%' ORDER BY Rating DESC LIMIT 10;")
result = con.fetchdf()
result

,DirectorName
0,Erik Ode
1,Kerry Brierley
2,Jesse Grce
3,Hans Scholz
4,Jorge Queiroga
5,Ronald Baez
6,Vidal
7,Scott O'Brien
8,Hillary Bradfield
9,Murray Debus


## *Q11: Who often made comedy movies?*

In [80]:
con.execute("SELECT DirectorName, COUNT(*) as comedy_movie_count FROM subset_df WHERE Genres LIKE '%Comedy%' GROUP BY DirectorName HAVING COUNT(*) > 1 ORDER BY comedy_movie_count DESC LIMIT 10;")
result = con.fetchdf()
result

,DirectorName,comedy_movie_count
0,Chuck O'Neil,2199
1,James Burrows,889
2,Doug Walker,824
3,Brad Jones,554
4,Joel Zwick,504
5,Gerren Keith,461
6,Dave Diomedi,459
7,James Widdoes,457
8,Alan Rafkin,454
9,Pamela Fryman,440


## *Q12: TV series have high ratings and high number of votes?*

In [94]:
con.execute("SELECT * FROM subset_df WHERE Type LIKE '%tvSeries%' AND Rating >= (SELECT AVG(Rating) FROM subset_df) AND NumberofVotes >= (SELECT AVG(NumberofVotes) FROM subset_df) ORDER BY Rating DESC;")
result = con.fetchdf()
result

,tconst,Type,Title,isAdult,startYear,endYear,Runtime,Genres,Rating,NumberofVotes,DirectorName
0,tt2298792,tvSeries,Alpha Bravo Charlie,0,1998,1998,40,Action,9.5,1810,Shoaib Mansoor
1,tt27548825,tvSeries,Fireflies: Parth Aur Jugnu,0,2023,\N,45,"Drama,Fantasy",9.5,5225,Hemant Gaba
2,tt8284230,tvSeries,The Last Drive-In with Joe Bob Briggs,0,2018,\N,125,"Comedy,Fantasy,Horror",9.5,1672,Austin Jennings
3,tt2910420,tvSeries,Drzavni Posao,0,2012,\N,10,Comedy,9.4,3531,Stojce Stoleski
4,tt15251018,tvSeries,Sabka Sai,0,2021,2021,42,"Biography,Drama",9.4,3375,Ajit Bhairavkar
...,...,...,...,...,...,...,...,...,...,...,...
773,tt7939800,tvSeries,The Undeclared War,0,2022,2022,47,"Drama,Thriller",6.9,4226,Peter Kosminsky
774,tt14935960,tvSeries,Takt Op. Destiny,0,2021,2021,24,"Action,Adventure,Animation",6.9,1865,Yûki Itô
775,tt16968320,tvSeries,RuPaul's Drag Race UK vs the World,0,2022,\N,70,"Game-Show,Reality-TV",6.9,1544,Tony Grech-Smith
776,tt17074560,tvSeries,The Rebel,0,2022,2022,50,Drama,6.9,3039,Mohammad Kart


## *Q13: Which TV series have high ratings and high number of votes released before 2000?*

In [98]:
con.execute("SELECT * FROM subset_df WHERE Type LIKE '%tvSeries%' AND Rating >= (SELECT AVG(Rating) FROM subset_df) AND NumberofVotes >= (SELECT AVG(NumberofVotes) FROM subset_df) ORDER BY Rating DESC;")
result = con.fetchdf()
# Convert the 'startYear' column to integer
result['startYear'] = result['startYear'].astype(int)
output = result[result['startYear'] < 2000]
output

,tconst,Type,Title,isAdult,startYear,endYear,Runtime,Genres,Rating,NumberofVotes,DirectorName
0,tt2298792,tvSeries,Alpha Bravo Charlie,0,1998,1998,40,Action,9.5,1810,Shoaib Mansoor
6,tt0077051,tvSeries,Matador,0,1978,1982,67,Drama,9.3,5056,Erik Balling
9,tt0389593,tvSeries,Chanakya,0,1990,\N,45,"Drama,History",9.3,2652,Chandra Prakash Dwivedi
10,tt0157217,tvSeries,Byomkesh Bakshi,0,1993,1997,40,"Adventure,Crime,Mystery",9.2,3055,Basu Chatterjee
26,tt0191747,tvSeries,Vruc vetar,0,1980,1980,55,Comedy,9.0,2581,Aleksandar Djordjevic
...,...,...,...,...,...,...,...,...,...,...,...
733,tt0064033,tvSeries,The Ant and the Aardvark,0,1969,1972,6,"Animation,Comedy,Family",7.0,1725,Friz Freleng
734,tt0096681,tvSeries,Prince Caspian and the Voyage of the Dawn Treader,0,1989,1989,30,"Adventure,Family,Fantasy",7.0,2377,Alex Kirby
753,tt0070996,tvSeries,Hong Kong Phooey,0,1974,1975,22,"Action,Adventure,Animation",6.9,3810,Charles Nichols
754,tt0081938,tvSeries,Spider-Man,0,1981,1982,30,"Action,Adventure,Animation",6.9,1716,Don Jurwich


## *Q14: Which movies have the longest duration and high rating?*

In [103]:
con.execute("SELECT Type,Title,Genres,Rating,Runtime FROM subset_df WHERE Rating > 7 ORDER BY Runtime DESC LIMIT 10;")
result = con.fetchdf()
result

,Type,Title,Genres,Rating,Runtime
0,movie,100,Drama,8.4,59460
1,video,The Longest Video on YouTube: 596.5 Hours,\N,8.0,35791
2,movie,Qw,Drama,7.3,10062
3,movie,Beijing 2003,Documentary,7.8,9000
4,movie,Ekalavya,"Drama,Thriller",9.5,5538
5,movie,Nieuwe Tieten,"Documentary,Drama",7.8,4080
6,tvSeries,White Deer Plain,"Drama,History",8.4,3825
7,tvSeries,Mundo CurioZoo,Family,7.6,3000
8,tvSeries,Kôya no surônin,"Action,Adventure,Drama",7.8,2925
9,movie,Five-Year Diary,Documentary,8.3,2160


## *Q14: Which excellent animation movies have adult content?*

In [108]:
con.execute("SELECT * FROM subset_df WHERE Genres LIKE '%Animation%' AND isAdult = 1 AND Rating > 8 ORDER BY Rating DESC;")
result = con.fetchdf()
result

,tconst,Type,Title,isAdult,startYear,endYear,Runtime,Genres,Rating,NumberofVotes,DirectorName
0,tt14469506,video,Dokidoki Little Ooyasan,1,2018,\N,120,"Adult,Animation",9.7,6,Ryuuzou Matano
1,tt19798926,video,Anehame: Ore no Hatsukoi ga Jisshi na Wake ga Nai,1,2021,\N,40,"Adult,Animation,Short",9.6,7,Toshihiro Watase
2,tt27033264,video,FandelTales - The Cursed Prince,1,2022,\N,17,"Adult,Animation,Fantasy",9.3,12,Derpixon
3,tt14442664,tvEpisode,Meru the Succubus OVA 2,1,2020,\N,12,"Adult,Animation,Comedy",9.3,44,Jack Pope
4,tt24043356,tvMiniSeries,Harem Camp!,1,2022,2022,7,"Adult,Animation,Romance",9.0,6,Toshihiro Watase
5,tt14774320,video,Kansen Sodom,1,2018,\N,20,"Action,Adult,Animation",9.0,6,Kazuya Komai
6,tt4301484,video,Lover-in-Law,1,2008,\N,30,"Adult,Animation,Romance",8.9,10,Tsukasa Kado
7,tt9341024,tvSeries,Itadaki! Seieki,1,2014,2014,23,"Adult,Animation,Comedy",8.9,73,Aoi Yuuno
8,tt14544994,tvEpisode,Meru the Succubus OVA 3,1,2021,\N,5,"Adult,Animation,Comedy",8.9,33,Jack Pope
9,tt15003770,video,Knight of Erin,1,2020,\N,80,"Adult,Adventure,Animation",8.8,9,Ayano Kunio


## *Q15: How many movies of James Cameron  in IMDb dataset?*

In [109]:
con.execute("SELECT * FROM df WHERE DirectorName LIKE '%James Cameron%';")
result = con.fetchdf()
result

,tconst,Type,Title,isAdult,startYear,endYear,Runtime,Genres,Rating,NumberofVotes,DirectorName
0,tt0088247,movie,The Terminator,0,1984,\N,107,"Action,Sci-Fi",8.1,888768,James Cameron
1,tt0090605,movie,Aliens,0,1986,\N,137,"Action,Adventure,Sci-Fi",8.4,734619,James Cameron
2,tt0096754,movie,The Abyss,0,1989,\N,140,"Adventure,Drama,Mystery",7.5,184091,James Cameron
3,tt0103064,movie,Terminator 2: Judgment Day,0,1991,\N,137,"Action,Sci-Fi",8.6,1127182,James Cameron
4,tt0111503,movie,True Lies,0,1994,\N,141,"Action,Comedy,Thriller",7.3,270477,James Cameron
5,tt0120338,movie,Titanic,0,1997,\N,194,"Drama,Romance",7.9,1223794,James Cameron
6,tt0297144,movie,Ghosts of the Abyss,0,2003,\N,61,"Documentary,Family",6.8,5895,James Cameron
7,tt0499549,movie,Avatar,0,2009,\N,162,"Action,Adventure,Fantasy",7.9,1348513,James Cameron
8,tt0554573,tvEpisode,Freak Nation,0,2002,\N,64,"Action,Drama,Mystery",8.5,501,James Cameron
9,tt15738638,video,Terminator 2: Judgment Day - Deleted Scenes,0,1993,\N,15,"Action,Sci-Fi,Short",8.5,307,James Cameron
